In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cc3d
import math
import cv2
from PIL import Image

import pandas as pd

from tqdm.notebook import tqdm

from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist

import scipy.ndimage as ndimage

import tifffile

from sklearn.cluster import KMeans

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:
# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


# ラベル画像を読み込む関数
def load_Y_gray(folder_path, thresh = None , normalize = True):
    image_files = []
    
    for file in os.listdir(folder_path):
        base , ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    images = np.zeros(
        (
            len(image_files) ,
            img.shape[0] ,
            img.shape[1] ,
            1
        ) ,
        np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = cv2.resize(image ,(img.shape[1] ,img.shape[0]))
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
    
    return images , image_files

In [ ]:
def create_label_imgs(label, mito, lamellar, tubular):
        target_mito = (mito==label).astype(int)
        target_tubular = np.where(
            target_mito>0,
            tubular[:,:,:,0],
            0
        ).astype(int)
        target_lamellar = np.where(
            target_mito>0,
            lamellar[:,:,:,0],
            0
        ).astype(int)


        target_mito_coords = target_mito.nonzero()
        z_min = np.min(target_mito_coords[0])
        z_max = np.max(target_mito_coords[0])
        y_min = np.min(target_mito_coords[1])
        y_max = np.max(target_mito_coords[1])
        x_min = np.min(target_mito_coords[2])
        x_max = np.max(target_mito_coords[2])
        
        cropped_target_mito = target_mito[z_min:z_max, y_min:y_max, x_min:x_max]
        cropped_target_tubular = target_tubular[z_min:z_max, y_min:y_max, x_min:x_max]
        cropped_target_lamellar = target_lamellar[z_min:z_max, y_min:y_max, x_min:x_max]
        
        merged_imgs = np.concatenate([
            cropped_target_mito[..., np.newaxis],
            cropped_target_tubular[..., np.newaxis],
            cropped_target_lamellar[..., np.newaxis],
        ], axis = -1)
        
        return merged_imgs
    
    
def create_cluster_imgs(target_imgs, size=5000):
    
    target_coords = convert_img2coords(target_imgs)
    cluster_num = np.sum(target_imgs)//size
    
    model = KMeans(n_clusters=cluster_num, random_state=0, init='random')
    model.fit(target_coords)
    
    clusters = model.predict(target_coords)
    clustered_imgs = convert_clustercoords2img(clusters, target_coords, target_imgs)
    
    return clustered_imgs


def convert_img2coords(imgs):
    
    _tuples = imgs.nonzero()
    
    _coords = np.zeros((_tuples[0].shape[0], 3))
    _coords[:,0] = _tuples[0]
    _coords[:,1] = _tuples[1]
    _coords[:,2] = _tuples[2]
    
    return _coords


def convert_clustercoords2img(cluster_coords, img_coords, imgs):
    
    new_imgs = np.zeros_like(imgs)
    for cluster_num, coords in zip(cluster_coords, img_coords):
        new_imgs[int(coords[0]), int(coords[1]), int(coords[2])] = cluster_num+1
        
    return new_imgs

        
    
def controller(GENE, CROP_NO):
    mito_imgs, _ = load_Y_gray(f'/home/suga/drobo/DeepLearningData/research_010_NIH3T3/{GENE}/annotations/{CROP_NO}/devided_mito/', normalize=False)
    lamellar_imgs, _ = load_Y_gray(f"/home/suga/drobo/DeepLearningData/research_010_NIH3T3/{GENE}/annotations/{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
    tubular_imgs, _ = load_Y_gray(f"/home/suga/drobo/DeepLearningData/research_010_NIH3T3/{GENE}/annotations/{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

    if GENE == "shCtrl_003" and CROP_NO == "cropped_002":
        mito_imgs = mito_imgs[:, :692, :2275]
        
    connectivity = 6 # 縦横下
    labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    
    for mito_label in tqdm(range(1, int(np.max(labels_mito_imgs)) + 1)):
        target_mito = (labels_mito_imgs==mito_label).astype(np.int64)
        
        if np.sum(target_mito) > 40:
            merge_labeled_imgs = create_label_imgs(mito_label, labels_mito_imgs, lamellar_imgs, tubular_imgs)
            
            cluster_imgs = create_cluster_imgs(merge_labeled_imgs[:,:,:,0], size=5000)
            
            df = pd.DataFrame(columns=["cluster_num", "mito", "tubular", "lamellar"], index=list( cluster_num for cluster_num in range(np.min(cluster_imgs), np.max(cluster_imgs)+1) ))
            for cluster_num in range(np.min(cluster_imgs), np.max(cluster_imgs)+1):
                cluster_mito = np.where(cluster_imgs==cluster_num, merge_labeled_imgs[:,:,:,0], 0)
                cluster_tubular = np.where(cluster_imgs==cluster_num, merge_labeled_imgs[:,:,:,1], 0)
                cluster_lamellar = np.where(cluster_imgs==cluster_num, merge_labeled_imgs[:,:,:,2], 0)
                df.at[cluster_num, "cluster_num"] = cluster_num
                df.at[cluster_num, "mito"] = np.sum(cluster_mito)
                df.at[cluster_num, "tubular"] = np.sum(cluster_tubular)
                df.at[cluster_num, "lamellar"] = np.sum(cluster_lamellar)
                
            out_dir = f'/home/suga/drobo/DeepLearningData/research_010_NIH3T3/{GENE}/data/{CROP_NO}/mito_subvolume_003_size5000/{str(mito_label).zfill(4)}'
            os.makedirs(out_dir, exist_ok = True)
            
            df.to_csv(f'/home/suga/drobo/DeepLearningData/research_010_NIH3T3/{GENE}/data/{CROP_NO}/mito_subvolume_003_size5000/{str(mito_label).zfill(4)}/count.csv')
            
            merge_labeled_imgs[:,:,:,0] = cluster_imgs
            merge_labeled_imgs[:,:,:,1] = merge_labeled_imgs[:,:,:,1] * cluster_imgs
            merge_labeled_imgs[:,:,:,2] = merge_labeled_imgs[:,:,:,2] * cluster_imgs
            tifffile.imwrite(f'/home/suga/drobo/DeepLearningData/research_010_NIH3T3/{GENE}/data/{CROP_NO}/mito_subvolume_003_size5000/{str(mito_label).zfill(4)}/mito_tubular_lamellar.tif', merge_labeled_imgs)
                
            

In [ ]:
controller("shCtrl_003", "cropped_001")
controller("shCtrl_003", "cropped_002")
controller("shCtrl_003", "cropped_003")
controller("shCtrl_003", "cropped_004")
controller("shCtrl_003", "cropped_005")

controller("shOPA1_003", "cropped_001")
controller("shOPA1_003", "cropped_002")
controller("shOPA1_003", "cropped_003")
controller("shOPA1_003", "cropped_004")
controller("shOPA1_003", "cropped_005")

In [ ]:
def merge_results(GENE, CROP_NO):
    
    INPUT_PARENT_PATH = f"/home/suga/drobo/DeepLearningData/research_010_NIH3T3/{GENE}/data/{CROP_NO}/mito_subvolume_003_size5000/"
    input_child_list = sorted(os.listdir(INPUT_PARENT_PATH))
    
    for input_child in input_child_list:
        if input_child == ".DS_Store":
            pass
        else:
            INPUT_PATH = f"{INPUT_PARENT_PATH}/{input_child}"
            target_df = pd.read_csv(f"{INPUT_PATH}/count.csv", index_col=0)
            
            if min(sum(target_df["tubular"]), sum(target_df["lamellar"])) == 0:
                pass
            else:
        
                try:
                    merge_df = pd.concat([merge_df, target_df])
                except:
                    merge_df = target_df
    
    return merge_df

In [ ]:
GENE = "shCtrl_003"
CROP_NO_list = ["cropped_001", "cropped_002", "cropped_003", "cropped_004", "cropped_005"]


for CROP_NO in CROP_NO_list:
    ctrl_cropped_df = merge_results(GENE, CROP_NO)
    ctrl_cropped_df["crop_no"] = CROP_NO
    
    try:
        ctrl_original_df = pd.concat([ctrl_original_df, ctrl_cropped_df])
    except:
        ctrl_original_df = ctrl_cropped_df
        
        
ctrl_non_zero_original_df = ctrl_original_df[ctrl_original_df["mito"] > 0]
ctrl_non_zero_original_df["tubular_ratio"] = ctrl_non_zero_original_df["tubular"] / ctrl_non_zero_original_df["mito"]
ctrl_non_zero_original_df["lamellar_ratio"] = ctrl_non_zero_original_df["lamellar"] / ctrl_non_zero_original_df["mito"]

In [ ]:
GENE = "shOPA1_003"
CROP_NO_list = ["cropped_001", "cropped_002", "cropped_003", "cropped_004", "cropped_005"]

for CROP_NO in CROP_NO_list:
    kd_cropped_df = merge_results(GENE, CROP_NO)
    kd_cropped_df["crop_no"] = CROP_NO
    
    try:
        kd_original_df = pd.concat([kd_original_df, kd_cropped_df])
    except:
        kd_original_df = kd_cropped_df
        
        
kd_non_zero_original_df = kd_original_df[kd_original_df["mito"] > 0]
kd_non_zero_original_df["tubular_ratio"] = kd_non_zero_original_df["tubular"] / kd_non_zero_original_df["mito"]
kd_non_zero_original_df["lamellar_ratio"] = kd_non_zero_original_df["lamellar"] / kd_non_zero_original_df["mito"]

In [ ]:
out_dir = "/home/suga/drobo/paper_data/2021_cristae/data_304_subvolume_analysis_of_tubular_lamellar"
os.makedirs(out_dir, exist_ok = True)

ctrl_non_zero_original_df.to_csv(f"{out_dir}/ctrl_raw.csv")
kd_non_zero_original_df.to_csv(f"{out_dir}/kd_raw.csv")